# 加载基本库

In [ ]:
import collections
import pandas as pd
import datetime
import ffmyh
import time
import math
# from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.metrics import log_loss
import xgboost
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
%run FeatureProcess.py
# from fastFM.datasets import make_user_item_regression
# from fastFM import als
from sklearn.metrics import mean_squared_error
import scipy.sparse as sp
import numpy as np

sklearn.__version__


# 加载数据项

In [ ]:
train_df_org = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_train_20180301.txt',sep=' ')
test_df_org = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_test_a_20180301.txt',sep=' ')



In [495]:
train_df_org.columns

train_df = train_df_org.copy().set_index("context_id")
test_df = test_df_org.copy().set_index("context_id")

test_df["is_trade"] = -1
train_df_prev = train_df.append(test_df)
# print len(train_df)
# print train_df[["is_trade"]]

# 预处理数据

In [496]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime("%Y-%m-%d")
def extract_time(x):
    d = datetime.fromtimestamp(x)
    return d.strftime("%Y-%m-%d %H:%M:%S")
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

train_df_prev['time'] = train_df_prev['context_timestamp'].apply(lambda x: extract_time(x))
train_df_prev['date'] = train_df_prev['context_timestamp'].apply(lambda x: extract_date(x))
train_df_prev['weekday'] = train_df_prev['context_timestamp'].apply(lambda x: extract_weekday(x))
train_df_prev['hour'] = train_df_prev['context_timestamp'].apply(lambda x: extract_hour(x))

m = {}
for idx, date in enumerate(sorted(train_df_prev['date'].unique())):
    m[date]=idx
print(m)

for idx, d in enumerate(sorted(train_df_prev['date'].unique())):
    train_df_prev["day"] = train_df_prev['date'].map(lambda x: m[x])

{'2018-09-24': 6, '2018-09-25': 7, '2018-09-20': 2, '2018-09-21': 3, '2018-09-22': 4, '2018-09-23': 5, '2018-09-19': 1, '2018-09-18': 0}


# 拆解复杂类型

In [497]:
from numpy import *
#简化list等复杂类型的结构
#item_category_list全展开
#item_property_list全展开取频率>0.05的数据
#predict_category_property会计算跟item prop和cate的余弦相似度
#具体是把两个list的数据拼成cate:-1和cate:prop两种方式拼成一个字符串再跟predict_category_property的数据计算相似度
def get_icl_map(df):
    print("get_icl_map ... ")
    dfX = df.copy()
    dfX = dfX['item_category_list'].str.split(';', expand=True)

#     s = sorted(list(dfX[0].unique()) + list(dfX[1].unique()) + list(dfX[2].unique()))
    m = {}
    for i in dfX[0].unique():
        if i == None:
            continue
        m[i] = "1"
    
    for i in dfX[1].unique():
        if i == None:
            continue
        m[i] = "2"
    
    for i in dfX[2].unique():
        if i == None:
            continue
        m[i] = "3"
    return m

def get_ipl_map(df):
    print("get_ipl_map ... ")
    df1 = df.copy()
    dfX = df1.copy()['item_property_list'].str.split(';')
    dfX = pd.DataFrame(dfX)
    
    m = collections.defaultdict(float)
    idx = 0
    for _, row in dfX.iterrows():
        for i in row[0]:
            m[i] += 1
    
    ll = len(dfX)
    for k,v in m.items():
        m[k] = v / ll
    return m

def cos_sim(a,b):
    if type(a) == type(b) == list:
        am,bm = {},{}
        for i in a:
            am[i] = 1
        for i in b:
            bm[i] = 1
        
        a = am
        b = bm
    
    up = 0.
    down = 0.
    for k in a:
        if k in b:
            up += (a[k]*b[k])
    down = math.sqrt(len(a))*math.sqrt(len(b))
    
    return up/down

def process_complex_types(dfX, icl_map, ipl_map):
    def filter_unless_cate(arr):
        ret = []
        for i in arr:
            if i in icl_map:
                ret.append(i)
        if len(ret) == 0:
            return None
        else:
            return ret
    
    def filter_unless_prop(arr):
        ret = []
        for i in arr:
            freq = ipl_map.get(i, 0.)
            if freq > 0.05:
                ret.append(i)
            else:
                ret.append(1)
        if len(ret) == 0:
            return None
        else:
            return unique_list(ret)
    
    def unique_list(arr):
        return list(set(arr))
    
    #{cate}:-1命中则为1分
    #{cate}:{prop}命中则为2分
    #后期优化权重
    def inner_product_recall_items(line):
        line = line.split("|")
        item_category_list = unique_list(line[0].split(";"))
        item_property_list = unique_list(line[1].split(";"))
        
        #删掉-1
        if "-1" in item_property_list:
            del item_property_list[item_property_list.index("-1")]
        
        
        #抽出预测的cate_list和prop_list
        line[2] = line[2].split(";")
        
        pitem_category_list_prop = {}
        pitem_category_list = []
        pitem_property_list = []
        for l in line[2]:
            l = l.split(":")
            
            if l[0] != -1:
                pitem_category_list.append(l[0])
            if len(l) >= 2 and l[1] != -1:
                l[1] = l[1].split(",")
                pitem_property_list.append(l[1])
                pitem_category_list_prop[l[0]] = l[1]
            if len(l) >= 3:
                print "FUCK?"

        
        #计算预测的cate相似度+prop相似度
        csim = cos_sim(item_category_list, pitem_category_list)
        psim = 0.
        if len(pitem_property_list) > 0:
            for i in pitem_property_list:
                psim += cos_sim(item_property_list, i)
            psim /= len(pitem_property_list)
        
        #统计category命中率
        hit_cate_rate = 0.
        hit_cate_sim = 1.
        if len(item_category_list) > 1:
            if len(item_category_list) == 2 and item_category_list[1] in pitem_category_list_prop:
                hit_cate_rate += 1
                hit_cate_sim *= (1+cos_sim(pitem_category_list_prop[item_category_list[1]], item_property_list))
            if len(item_category_list) == 3 and item_category_list[2] in pitem_category_list_prop:
                hit_cate_rate += 1
                hit_cate_sim *= (1+cos_sim(pitem_category_list_prop[item_category_list[2]], item_property_list))
            
            hit_cate_rate /= (len(item_category_list) - 1)
            
            
        
        
        predict_richness = len(set(pitem_category_list))
        item_property_richness = len(set(item_property_list))
        return [csim, psim, predict_richness, item_property_richness, hit_cate_rate, hit_cate_sim]
            
        
            
        
        
        
    print("processing predict_category_property ...")
#     dfX['predict_category_property'] = dfX['predict_category_property'].str.split(';').map(lambda x: [i.split(":")[0] for i in x]).map(filter_unless_cate)
    
    dfX['tmp'] = dfX['item_category_list']+"|"+dfX['item_property_list']+"|"+dfX['predict_category_property']
    dfX['tmp'] = dfX['tmp'].map(inner_product_recall_items)
    
    dfX['category_sim'] = dfX['tmp'].map(lambda x: x[0])
    dfX['property_sim'] = dfX['tmp'].map(lambda x: x[1])
    dfX['predict_richness'] =  dfX['tmp'].map(lambda x: x[2])
    dfX['item_property_richness'] = dfX['tmp'].map(lambda x: x[3])
    dfX['hit_cate_cnt'] = dfX['tmp'].map(lambda x: x[4])
    dfX['hit_cate_sim'] = dfX['tmp'].map(lambda x: x[5])
    dfX.drop("tmp")
    
    print("processing item_property_list ...")
    dfX['item_property_list'] = dfX['item_property_list'].str.split(';').map(filter_unless_prop)
    
    print("processing item_category_list ...")
    dfX['item_category_list'] = dfX['item_category_list'].str.split(';')
    
    print "generating item_category_1, item_category_2 ..."
#     dfX['item_category_list01'] = dfX['item_category_list'].map(lambda x:x[0] if x != None and len(x) > 0 else None)
    dfX['item_category_1'] = dfX['item_category_list'].map(lambda x:x[1] if x != None and len(x) > 1 else None)
    dfX['item_category_2'] = dfX['item_category_list'].map(lambda x:x[2] if x != None and len(x) > 2 else None)
    
    return dfX



# aaa = process_complex_types(train_df.copy(), get_icl_map(train_df), get_ipl_map(train_df))

train_df_prev = process_complex_types(train_df_prev, get_icl_map(train_df_prev), get_ipl_map(train_df_prev))


get_icl_map ... 
get_ipl_map ... 
processing predict_category_property ...
processing item_property_list ...
processing item_category_list ...
generating item_category_1, item_category_2 ...


## 观察复杂类型效果

In [ ]:
tmp_df = train_df_prev.copy()
pd.set_option('max_colwidth', 500)
tmp_df[tmp_df["is_trade"] == 1][["hit_cate_cnt", "hit_cate_sim", "category_sim", "property_sim", "predict_richness","item_property_richness"]].describe()


In [ ]:
a,b,c,d,e,f=[],[],[],[],[],[]
for i in xrange(100):
    mean1 = dict(tmp_df[tmp_df["is_trade"] == 0].sample(9021, random_state=i)[["predict_richness"]].describe())["predict_richness"]["mean"]
    mean2 = dict(tmp_df[tmp_df["is_trade"] == 0].sample(9021, random_state=i)[["item_property_richness"]].describe())["item_property_richness"]["mean"]
    mean3 = dict(tmp_df[tmp_df["is_trade"] == 0].sample(9021, random_state=i)[["category_sim"]].describe())["category_sim"]["mean"]
    mean4 = dict(tmp_df[tmp_df["is_trade"] == 0].sample(9021, random_state=i)[["property_sim"]].describe())["property_sim"]["mean"]
    mean5 = dict(tmp_df[tmp_df["is_trade"] == 0].sample(9021, random_state=i)[["hit_cate_cnt"]].describe())["hit_cate_cnt"]["mean"]
    mean6 = dict(tmp_df[tmp_df["is_trade"] == 0].sample(9021, random_state=i)[["hit_cate_sim"]].describe())["hit_cate_sim"]["mean"]

    a.append(mean1)
    b.append(mean2)
    c.append(mean3)
    d.append(mean4)
    e.append(mean5)
    f.append(mean6)
    
print max(a),min(a)
print max(b),min(b)
print max(c),min(c)
print max(d),min(d)
print max(e), min(e)
print max(f), min(f)

## 观察复杂类型效果

In [ ]:
tmp_df = train_df_prev.copy()
tmp_df = tmp_df[tmp_df["day"] == 1]
d1 = tmp_df[tmp_df["hour"] >= 18]
d1 = d1[d1["hour"] <= 21]
d2 = tmp_df[tmp_df["hour"]==22]
d1 = d1[["shop_id","instance_id"]].groupby(["shop_id"])['instance_id'].count().to_dict() 
d2["accesscnt"] = d2["shop_id"].map(lambda x: d1.get(x, 0))

In [ ]:
d2[d2["is_trade"] == 1][["accesscnt"]].describe()

In [ ]:
a = []
for i in xrange(1000):
    a.append(dict(d2[d2["is_trade"] == 0].sample(83,random_state=i)[["accesscnt"]].describe())["accesscnt"]["mean"])
print max(a),min(a)

## 页码交易率观察

In [ ]:
tmp_df = train_df_prev[train_df_prev["day"]<7].copy()
tmp_df = tmp_df[["context_page_id","is_trade"]]
tmp_df["context_page_id"] = tmp_df["context_page_id"] - 4000
 
print "页码 | 数量 | 交易率"
print "- | :-: | -: "
for i in xrange(1,21):
    a = tmp_df[tmp_df["context_page_id"]==i]
    mean = dict(a[["is_trade"]].describe())["is_trade"]["mean"]
    print i, "|", len(a), "|", mean

页码 | 数量 | 交易率
- | :-: | -: 
1 | 267486 | 0.0196122413883
2 | 43751 | 0.0211195172682
3 | 31455 | 0.0186297885869
4 | 24088 | 0.0190966456327
5 | 18821 | 0.0177992667765
6 | 15096 | 0.0172231054584
7 | 12355 | 0.0184540671793
8 | 10396 | 0.0179876875721
9 | 9014 | 0.0157532726869
10 | 7472 | 0.0163276231263
11 | 6372 | 0.0138104205901
12 | 5801 | 0.0151697983106
13 | 4892 | 0.0153311529027
14 | 4233 | 0.0132293881408
15 | 3823 | 0.014648182056
16 | 3367 | 0.014553014553
17 | 2999 | 0.0113371123708
18 | 2535 | 0.0130177514793
19 | 2242 | 0.0107047279215
20 | 1940 | 0.0144329896907

## 页码交易率观察

## 统计一个用户重复看某个商品的次数+一个用户重复看某类商品的次数

In [513]:
# cnt_user_item_review = train_df_prev[["user_id", "item_id", "instance_id"]].groupby(["user_id", "item_id"])['instance_id'].count().to_dict() 
# cnt_user_cate_review = train_df_prev[["user_id", "item_category_1", "instance_id"]].groupby(["user_id", "item_category_1"])['instance_id'].count().to_dict()  
# cnt_user_brand_review = train_df_prev[["user_id", "item_brand_id", "instance_id"]].groupby(["user_id", "item_brand_id"])['instance_id'].count().to_dict()  
        
# f1 = set_review_cnt("user_id", "item_id", cnt_user_item_review)
# f2 = set_review_cnt("user_id", "item_category_1", cnt_user_cate_review)
# f3 = set_review_cnt("user_id", "item_brand_id", cnt_user_brand_review)

# train_df_prev = train_df_prev.sort_values(by="context_timestamp")
# train_df_prev["item_review_cnt"] = train_df_prev[["user_id", "item_id"]].apply(f1, axis=1)
# train_df_prev["cate_review_cnt"] = train_df_prev[["user_id", "item_category_1"]].apply(f2, axis=1)
# train_df_prev["brand_review_cnt"] = train_df_prev[["user_id", "item_brand_id"]].apply(f3, axis=1)

class set_review_cnt:
    def __init__(self, key1, key2, cnt_k1_k2_review):
        self.key1 = key1
        self.key2 = key2
        self.tmp = collections.defaultdict(int)
        self.cnt_k1_k2_review = cnt_k1_k2_review
    def __call__(self, x):
        val1,val2 = x[self.key1], x[self.key2]
        vk = (val1, val2)
        if val2 < 0:
            return np.nan 
        if val2 == None:
            return np.nan

        cnt = self.cnt_k1_k2_review[vk]
        if cnt == 0:
            return 0
        else:
            
            k = "%s_%s" % (val1, val2)
            ret = self.tmp[k]
            self.cnt_k1_k2_review[vk] -= 1
            self.tmp[k] += 1
            return ret
        
def use_feautre_review_cnt(df, feature, new_feature):
    
    cnt_feature_review = df[["user_id", feature, "instance_id"]].groupby(["user_id", feature])['instance_id'].count().to_dict() 
    f = set_review_cnt("user_id", feature, cnt_feature_review)
    
    df = df.sort_values(by="context_timestamp")
    df[new_feature] = df[["user_id", feature]].apply(f, axis=1)
    
    return df

    

    
tmp_df = train_df_prev.copy()

features = [
    ["item_id", "item_review_cnt"],
    ["item_category_1", "cate_review_cnt"],
#     ["item_category_2", "cate2_review_cnt"],
    ["item_brand_id", "brand_review_cnt"],
    ["item_price_level", "price_review_cnt"],
    ["item_sales_level", "sales_review_cnt"],
    ["item_pv_level", "pv_review_cnt"],
    ["item_collected_level", "collected_review_cnt"],
    ["item_city_id", "city_review_cnt"]
]

show_features = []
for f in features:
    print "caling %s into %s ..." % (f[0], f[1])
    tmp_df = use_feautre_review_cnt(tmp_df, f[0], f[1])
    show_features.append(f[1])
    

caling item_id into item_review_cnt ...
caling item_category_1 into cate_review_cnt ...
caling item_category_2 into cate2_review_cnt ...
caling item_brand_id into brand_review_cnt ...
caling item_price_level into price_review_cnt ...
caling item_sales_level into sales_review_cnt ...
caling item_pv_level into pv_review_cnt ...
caling item_collected_level into collected_review_cnt ...
caling item_city_id into city_review_cnt ...


In [577]:
train_df_prev = tmp_df
for f in show_features:
    print "\n", f
    for v in train_df_prev[f].unique():
        print v, len(train_df_prev[train_df_prev[f] == v])
    print '-' * 30
    print ""
cnt_feautre_review = show_features
train_df_prev[show_features].describe()


# cnt_user_item_review = train_df_prev[["user_id", "item_id", "instance_id"]].groupby(["user_id", "item_id"])['instance_id'].count().to_dict() 

# ma = collections.defaultdict(float)
# mb = collections.defaultdict(float)
# for k,v in cnt_user_item_review.items():
#     ma[k[1]] += 1
#     if v > 0:
#         mb[k[1]] += 1
# # print max(cnt_user_item_review.values())
# # print max(mb.values())
# for k,v in ma.items():
#     print k,mb[k]/v


item_review_cnt
0 461737
1 29021
2 4257
3 999
4 278
5 97
6 46
7 21
8 14
9 11
10 9
11 7
12 4
13 4
14 3
15 1
------------------------------


cate_review_cnt
0 275416
1 104501
2 48942
3 25613
4 14554
5 8824
6 5573
7 3602
8 2422
9 1696
10 1258
11 895
12 669
13 499
14 384
15 303
16 241
17 191
18 154
19 131
20 102
21 78
22 64
23 52
24 46
25 37
26 32
27 30
28 27
29 24
30 19
31 16
32 15
33 13
34 11
35 11
36 10
37 8
38 7
39 6
40 6
41 5
42 5
43 2
44 2
45 2
46 2
47 2
48 2
49 2
50 1
51 1
52 1
------------------------------


cate2_review_cnt
nan 0
0.0 1519
1.0 360
2.0 117
3.0 47
4.0 23
5.0 14
6.0 9
7.0 3
8.0 2
9.0 2
10.0 1
11.0 1
12.0 1
13.0 1
14.0 1
15.0 1
16.0 1
17.0 1
18.0 1
19.0 1
20.0 1
21.0 1
------------------------------


brand_review_cnt
0.0 428996
1.0 49250
2.0 11154
3.0 3638
4.0 1433
5.0 670
nan 0
6.0 338
7.0 188
8.0 104
9.0 77
10.0 54
11.0 33
12.0 21
13.0 18
14.0 12
15.0 9
16.0 8
17.0 5
18.0 5
19.0 3
20.0 2
------------------------------


price_review_cnt
0 307653
1 99361
2 41407
3

,item_review_cnt,cate_review_cnt,cate2_review_cnt,brand_review_cnt,price_review_cnt,sales_review_cnt,pv_review_cnt,collected_review_cnt,city_review_cnt
count,496509.000000,496509.000000,2108.000000,496018.000000,496509.000000,495561.000000,496509.000000,496509.000000,496226.000000
mean,0.086780,1.109394,0.565465,0.198612,0.831550,0.319499,0.367798,0.318697,1.582245
std,0.365856,2.142329,1.553043,0.647292,1.697039,0.800378,0.878178,0.796520,1.281980
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000
max,15.000000,52.000000,21.000000,20.000000,49.000000,27.000000,26.000000,24.000000,11.000000


# 转化属性增加trade_rate + trade_pv（按前一天）

In [518]:
# 统计信息附加

def calcTVTransform(df, key, key_y, filter_src, filter_dst, smoothing = 5, mean0=None):
    if mean0 is None:
        #计算目标的平均值做平缓用
        mean0 = df.ix[filter_src, key_y].mean()
        print("mean0:", mean0)
    
    #取出key的所有值
    df['_key1'] = df[key].astype('category').values.codes
    
    
    #取出用于计算的源（后面聚合掉就没有顺序可言了）
    df_key1_y = df.ix[filter_src, ['_key1', key_y]]
    
    #根据key的取值去聚合key_y的总数和总和，用户计算rate和count
    grp1 = df_key1_y.groupby(['_key1'])
    sum1 = grp1[key_y].aggregate(np.sum)
    cnt1 = grp1[key_y].aggregate(np.size)
    
    vn_sum = 'sum_' + key
    vn_cnt = 'cnt_' + key
    
    #取出dst（带序列）的所有key
    v_codes = df.ix[filter_dst, '_key1']
    
    #得到_sum,_cnt，按dst的序列
    _sum = sum1[v_codes].values
    _cnt = cnt1[v_codes].values
    _cnt[np.isnan(_sum)] = 0    
    _sum[np.isnan(_sum)] = 0
    
    r = {}
    r['exp'] = (_sum + smoothing * mean0)/(_cnt + smoothing)
    r['cnt'] = _cnt
    return r


In [519]:
#计算前一天的交易率set到下一天，第0天用回自己
# tmp = train_df.copy()
tmp = train_df_prev

add_count = False
# window = 2

exp = "exp_d_"
cnt = "cnt_d_"


exp_numerical = {}
cnt_numerical = {}

In [520]:

k = "day"
exp_k = "exp_d_day"
for day in xrange(0,7):
    cal_day = day
    set_day = day

    print("cal %s trade_rate cnt %s set to %s" % (k,cal_day, set_day))

    #start_d - day(不含day)用于计算，结果赋值到day上
    days1 = (tmp.day.values == cal_day)
    days2 = (tmp.day.values == set_day)
    ret = calcTVTransform(tmp, k, 'is_trade', days1, days2, 0)

    tmp.loc[tmp.day.values == day, exp_k] = ret["exp"]

    exp_numerical[exp_k]=1

tmp.loc[tmp.day.values == 7, exp_k] = 0.0169

tmp[[exp_k, "day"]]

cal day trade_rate cnt 0 set to 0
('mean0:', 0.020020953646445547)


/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


cal day trade_rate cnt 1 set to 1
('mean0:', 0.019681098532376536)
cal day trade_rate cnt 2 set to 2
('mean0:', 0.01936040475528975)
cal day trade_rate cnt 3 set to 3
('mean0:', 0.019312068989733002)
cal day trade_rate cnt 4 set to 4
('mean0:', 0.018867648350361546)
cal day trade_rate cnt 5 set to 5
('mean0:', 0.017276071305058635)
cal day trade_rate cnt 6 set to 6
('mean0:', 0.016910189651869526)


,exp_d_day,day
context_id,,
7943330529831398173,0.020021,0
8211008059339466280,0.020021,0
566319850199326447,0.020021,0
5650748176010570658,0.020021,0
4495862064771683624,0.020021,0
4357642332875457175,0.020021,0
7983113595161510967,0.020021,0
9206379698438307321,0.020021,0
8223959869957177585,0.020021,0


In [521]:


#此处应该处理按天为梯度的数据
for k in ['item_price_level',
'item_sales_level',
'item_collected_level',
'item_pv_level',
'user_gender_id',
'user_age_level',
'user_occupation_id',
'user_star_level',
'context_page_id',
'shop_review_num_level',
'shop_star_level',
         

"item_brand_id",
"item_city_id",
"item_id",
"shop_id",
# "user_id",
          
"item_brand_id",
"item_city_id"
         ]:
    exp_k = exp+k
    cnt_k = cnt+k
    for day in xrange(0,8):
#         start_d = max(day - window, 0)
#         end_d = max(day - 1,0)
        cal_day = max(day - 1, 0)
        set_day = day

        print("cal %s trade_rate cnt %s set to %s" % (k,cal_day, set_day))
        
        #start_d - day(不含day)用于计算，结果赋值到day上
        days1 = (tmp.day.values == cal_day)
        days2 = (tmp.day.values == set_day)
        ret = calcTVTransform(tmp, k, 'is_trade', days1, days2)
            
        tmp.loc[tmp.day.values == day, exp_k] = ret["exp"]
        
        exp_numerical[exp_k]=1
        if add_count:
            cnt_numerical[cnt_k]=1
            tmp.loc[tmp.day.values == day, cnt_k] = ret["cnt"]


cal item_price_level trade_rate cnt 0 set to 0
('mean0:', 0.020020953646445547)
cal item_price_level trade_rate cnt 0 set to 1
('mean0:', 0.020020953646445547)
cal item_price_level trade_rate cnt 1 set to 2
('mean0:', 0.019681098532376536)
cal item_price_level trade_rate cnt 2 set to 3
('mean0:', 0.01936040475528975)
cal item_price_level trade_rate cnt 3 set to 4
('mean0:', 0.019312068989733002)
cal item_price_level trade_rate cnt 4 set to 5
('mean0:', 0.018867648350361546)
cal item_price_level trade_rate cnt 5 set to 6
('mean0:', 0.017276071305058635)
cal item_price_level trade_rate cnt 6 set to 7
('mean0:', 0.016910189651869526)
cal item_sales_level trade_rate cnt 0 set to 0
('mean0:', 0.020020953646445547)
cal item_sales_level trade_rate cnt 0 set to 1
('mean0:', 0.020020953646445547)
cal item_sales_level trade_rate cnt 1 set to 2
('mean0:', 0.019681098532376536)
cal item_sales_level trade_rate cnt 2 set to 3
('mean0:', 0.01936040475528975)
cal item_sales_level trade_rate cnt 3 set 

cal item_id trade_rate cnt 0 set to 1
('mean0:', 0.020020953646445547)
cal item_id trade_rate cnt 1 set to 2
('mean0:', 0.019681098532376536)
cal item_id trade_rate cnt 2 set to 3
('mean0:', 0.01936040475528975)
cal item_id trade_rate cnt 3 set to 4
('mean0:', 0.019312068989733002)
cal item_id trade_rate cnt 4 set to 5
('mean0:', 0.018867648350361546)
cal item_id trade_rate cnt 5 set to 6
('mean0:', 0.017276071305058635)
cal item_id trade_rate cnt 6 set to 7
('mean0:', 0.016910189651869526)
cal shop_id trade_rate cnt 0 set to 0
('mean0:', 0.020020953646445547)
cal shop_id trade_rate cnt 0 set to 1
('mean0:', 0.020020953646445547)
cal shop_id trade_rate cnt 1 set to 2
('mean0:', 0.019681098532376536)
cal shop_id trade_rate cnt 2 set to 3
('mean0:', 0.01936040475528975)
cal shop_id trade_rate cnt 3 set to 4
('mean0:', 0.019312068989733002)
cal shop_id trade_rate cnt 4 set to 5
('mean0:', 0.018867648350361546)
cal shop_id trade_rate cnt 5 set to 6
('mean0:', 0.017276071305058635)
cal shop

In [522]:
exp_numerical = exp_numerical.keys()
cnt_numerical = cnt_numerical.keys()

print(exp_numerical)
print(cnt_numerical)



['exp_d_user_age_level', 'exp_d_item_pv_level', 'exp_d_shop_review_num_level', 'exp_d_item_sales_level', 'exp_d_user_occupation_id', 'exp_d_item_city_id', 'exp_d_item_brand_id', 'exp_d_user_star_level', 'exp_d_item_price_level', 'exp_d_day', 'exp_d_item_id', 'exp_d_shop_star_level', 'exp_d_item_collected_level', 'exp_d_user_gender_id', 'exp_d_context_page_id', 'exp_d_shop_id']
[]


In [582]:
print "RECOVER"
train_df = train_df_prev.copy()

RECOVER


# 标记训练用的列

In [524]:
#标记训练数据

import copy
target="is_trade"
                          
                          
                        
categorical=[  
                'item_price_level',
                'item_sales_level',
                'item_collected_level',
                'item_pv_level',
                'user_gender_id',
                'user_age_level',
                'user_occupation_id',
                'user_star_level',
                'context_page_id',
                'shop_review_num_level',
                'shop_star_level',
                "weekday",
                "hour",
    
                'item_category_1',
                'item_category_2',
                
                "item_brand_id",
                "item_city_id",
                "item_id",
                "shop_id",
    
            ]+cnt_feautre_review

numerical=[     'shop_review_positive_rate',
                'shop_score_service',
                'shop_score_delivery',
                'shop_score_description',
#                 'predict_category_property',
                'predict_richness',
                'item_property_richness'
                   
#               ]
          ]+exp_numerical

listype = [
    'item_property_list', 
#     'item_category_list'
]

                    
            

cols_tool = filter_on_cols(target, categorical, numerical, listype)

In [525]:
train_df[cols_tool.get_raw_numerical_cols()].describe()

,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,predict_richness,item_property_richness,exp_d_user_age_level,exp_d_item_pv_level,exp_d_shop_review_num_level,exp_d_item_sales_level,exp_d_user_occupation_id,exp_d_item_city_id,exp_d_item_brand_id,exp_d_user_star_level,exp_d_item_price_level,exp_d_day,exp_d_item_id,exp_d_shop_star_level,exp_d_item_collected_level,exp_d_user_gender_id,exp_d_context_page_id,exp_d_shop_id
count,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000
mean,0.994833,0.971131,0.970495,0.974870,4.810191,34.235279,0.019208,0.019182,0.019205,0.019192,0.019196,0.019149,0.019156,0.019191,0.019127,0.018794,0.019465,0.019197,0.019203,0.019201,0.019204,0.019042
std,0.011875,0.023607,0.023567,0.024823,1.795871,10.100147,0.003049,0.002667,0.003738,0.005930,0.001746,0.007948,0.017160,0.002817,0.008326,0.001128,0.023115,0.003774,0.003000,0.001897,0.002717,0.020898
min,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,6.000000,0.000985,0.000722,0.000925,0.000521,0.000985,0.000455,0.000229,0.000985,0.000991,0.016900,0.000287,0.000531,0.000545,0.010104,0.004171,0.000237
25%,0.992780,0.966360,0.965677,0.969282,3.000000,28.000000,0.017564,0.017939,0.017704,0.015125,0.018679,0.013794,0.006876,0.018385,0.014194,0.017276,0.004033,0.016705,0.017904,0.018869,0.018735,0.004389
50%,1.000000,0.972347,0.971579,0.978493,5.000000,33.000000,0.019105,0.019256,0.018837,0.019286,0.019032,0.016974,0.017874,0.019564,0.016366,0.019312,0.011792,0.018953,0.019162,0.019397,0.019370,0.012949
75%,1.000000,0.977822,0.976966,0.983626,6.000000,38.000000,0.020800,0.020019,0.021641,0.022024,0.019854,0.022694,0.024959,0.020864,0.023363,0.019681,0.025886,0.022550,0.020535,0.019800,0.020518,0.027273
max,1.000000,1.000000,1.000000,1.000000,14.000000,99.000000,0.027449,0.182390,0.084623,0.037044,0.022760,0.189505,0.231820,0.054228,0.109656,0.020021,0.299543,0.122234,0.035202,0.023104,0.026027,0.262301


# 平滑处理连续型数据最后对复杂类型做onehot编码

In [526]:
#数据的预处理
#double系列(例如好评率)的数据存在一个问题，没有考虑到评论量，所以乘了一个评论level，类似 好评率*评论数(分桶了) = 好评量(分桶了)
#fillempty把-1设置成-0.01，配合mov2pos使用
#mov2pos把有<0的数据都集体-min，如果一个数据是0-1，有-1出现的时候，上面设置成了-0.01，所以这列数据真实是-0.01 - 1,然后我会集体-(-0.01)
#norm用了最大值最小值norm
%run FeatureProcess.py
from sklearn import preprocessing

def bucketizer(df, key, bins):
    le = preprocessing.LabelEncoder()
    prefix="*BUK*_"
    df[prefix+key] = pd.cut(df[key].values, bins=bins, retbins=False)
    df[prefix+key] = le.fit_transform(df[prefix+key].copy())
    return df


def layering(df, key, by_key):
    uniqv = df[by_key].unique()
    keys = []
    for v in uniqv:
        indexer = (df[by_key] == v)
        new_key = "*%sLAY*_%s" % (str(v), key)
        df.loc[indexer, new_key] = df[indexer][key]
        keys.append(new_key)
    return df, keys

# #分桶+打散
# print("bucketizer ... ")
# train_df = bucketizer(train_df, "shop_review_num_level", 5)

# print("layering ... ")
# new_n = []
# for n in numerical:
#     train_df, keys = layering(train_df, n, '*BUK*_shop_review_num_level')
#     new_n += keys



featProc = FeatureProcess(target=target, categorical=categorical, numerical=numerical, listype = listype)
# for n in numerical:
#     featProc.addLayeringOrgKeys(n)
# train_df["shop_review_num_level"] = train_df["shop_review_num_level"]
# train_df["shop_review_positive_rate"] = train_df["shop_review_positive_rate"]*train_df["shop_review_num_level"]
# train_df["shop_score_service"] = train_df["shop_score_service"]*train_df["shop_review_num_level"]
# train_df["shop_score_delivery"] = train_df["shop_score_delivery"]*train_df["shop_review_num_level"]
# train_df["shop_score_description"] = train_df["shop_score_description"]*train_df["shop_review_num_level"]

train_df = featProc.fillempty(train_df, -0.01)
# train_df = featProc.mov2pos(train_df)
train_df = featProc.mov2mean(train_df)
train_df = featProc.norm(train_df)
train_df[cols_tool.get_raw_numerical_cols()].describe()


,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,predict_richness,item_property_richness,exp_d_user_age_level,exp_d_item_pv_level,exp_d_shop_review_num_level,exp_d_item_sales_level,exp_d_user_occupation_id,exp_d_item_city_id,exp_d_item_brand_id,exp_d_user_star_level,exp_d_item_price_level,exp_d_day,exp_d_item_id,exp_d_shop_star_level,exp_d_item_collected_level,exp_d_user_gender_id,exp_d_context_page_id,exp_d_shop_id
count,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000,496509.000000
mean,0.982014,0.865794,0.827843,0.883320,0.293092,0.303605,0.688612,0.101611,0.218402,0.511220,0.836325,0.098886,0.081723,0.341940,0.166898,0.606917,0.064086,0.153373,0.538369,0.699759,0.687828,0.071759
std,0.032251,0.043904,0.054594,0.056408,0.138144,0.108604,0.115209,0.014680,0.044658,0.162354,0.080191,0.042041,0.074096,0.052908,0.076621,0.361303,0.077242,0.031012,0.086564,0.145956,0.124316,0.079743
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.974731,0.842328,0.798142,0.856064,0.153846,0.236559,0.626489,0.094769,0.200471,0.399855,0.812573,0.070562,0.028698,0.326793,0.121504,0.120499,0.012521,0.132899,0.500887,0.674225,0.666383,0.015845
50%,1.000000,0.870375,0.832816,0.899184,0.307692,0.290323,0.684715,0.102018,0.214012,0.513787,0.828809,0.087382,0.076188,0.348955,0.141488,0.772863,0.038448,0.151369,0.537175,0.714830,0.695428,0.048508
75%,1.000000,0.896039,0.864508,0.923249,0.384615,0.344086,0.748742,0.106222,0.247507,0.588760,0.866525,0.117639,0.106783,0.373371,0.205879,0.891105,0.085545,0.180926,0.576776,0.745833,0.747943,0.103168
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# #onehot没啥好说的，就是onehot，toOneHotList实现有点不同分开了而已


# # train_df = featProc.toOneHot(train_df)
# train_df = featProc.toOneHotList(train_df)

# # train_df = featProc.cacheRun(featProc.toOneHot, train_df)

# for c in train_df.columns:
#     print(c)

# print train_df.columns

# 定义模型函数

In [573]:
%run ffmyh.py

    
# def train_and_test_combine(train_df, test_df, res_df):
    
#     X_train = train_df.copy()
#     del X_train['is_trade']
#     y_train = train_df['is_trade']
    
#     X_test = test_df.copy()
#     del X_test['is_trade']
#     y_test = test_df['is_trade']
    
    
    
#     model = xgboost.XGBClassifier(nthread=7,max_depth=5)
#     model.fit(X_train, y_train)
    
#     y_prev_predict_train = pd.DataFrame(model.predict_proba(X_train))
#     y_prev_predict_train["idx"] = X_train.index
#     y_prev_predict_train = y_prev_predict_train.set_index("idx")
#     X_train["y_prev_predict"] = y_prev_predict_train[1]
    
#     y_prev_predict_test = pd.DataFrame(model.predict_proba(X_test))
#     y_prev_predict_test["idx"] = X_test.index
#     y_prev_predict_test = y_prev_predict_test.set_index("idx")
#     X_test["y_prev_predict"] = y_prev_predict_test[1]
    
    
# #     X_train = X_train.fillna(0)
# #     X_test = X_test.fillna(0)
    
    
#     model = LogisticRegression(max_iter=1000)
#     model.fit(X_train, y_train)
    
    
#     train_lls = log_loss(y_train,model.predict_proba(X_train))
#     test_lls = log_loss(y_test,model.predict_proba(X_test))
#     del train_df
#     del test_df
    
#     del X_train
#     del y_train
#     del X_test
#     del y_test
    
#     return train_lls, test_lls
    

def train_and_test_xgboost(train_df, test_df, res_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
#     model = xgboost.XGBClassifier(reg_lambda=1.5,learning_rate=0.1,reg_alpha=0,nthread=7,n_estimators=100,max_depth=10)
    model = xgboost.XGBClassifier(subsample=0.9,colsample_bytree=0.9,n_estimators=500,max_depth=7,nthread=3)

#     model = xgboost.XGBClassifier(subsample=0.9,colsample_bytree=0.9,n_estimators=50,max_depth=3,nthread=1)
    model.fit(X_train, y_train)
    fig, ax = plt.subplots(figsize=(12,18))
    xgboost.plot_importance(model,ax=ax)
    plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("xgboost_res.csv",index=False,sep=' ')
        
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_lr(train_df, test_df, res_df):    
    X_train = train_df.copy().fillna(0)
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy().fillna(0)
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
    
    model = LogisticRegression(C=1.0, n_jobs=7,tol=1e-6, max_iter=2000)
    model.fit(X_train.values, y_train)
#     for idx, val in enumerate(list(X_train.columns)):
#         print("%s=%s" %(val,list(model.coef_[0])[idx]))


    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        res_df = res_df.copy().fillna(0)
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("lr_res.csv",index=False,sep=' ')
    
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls

def train_and_test_randomforest(train_df, test_df, res_df):    
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
#     rf = RandomForestClassifier(n_estimators=32, max_depth=40, min_samples_split=100, min_samples_leaf=10,  criterion='entropy',
#                         max_features=8, verbose = 1,  bootstrap=False, n_jobs=10)
#     RandomForestClassifier(criterion='entropy',n_estimators=100,n_jobs=15)

    model = RandomForestClassifier(random_state=9,n_jobs=7, n_estimators=100, max_depth=10, min_samples_split=100, min_samples_leaf=10,  criterion='entropy', max_features=8, verbose = 1,  bootstrap=False)
    model.fit(X_train, y_train)
    
    ft_w = pd.DataFrame(model.feature_importances_, columns=["weights"], index = list(X_train.columns))
    print ft_w.sort_values("weights", ascending=False)
#     fig, ax = plt.subplots(figsize=(12,18))
#     xgboost.plot_importance(model,ax=ax)
#     plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("randomforest_res.csv",index=False,sep=' ')
        
        
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split( X_test, y_test, test_size=0.7, shuffle=True, random_state=6)
    test_a_lls = log_loss(y_val_a,model.predict_proba(X_val_a))
    test_b_lls = log_loss(y_val_b,model.predict_proba(X_val_b))
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls, test_a_lls, test_b_lls

def train_and_test_gbdt(train_df, test_df, res_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    model = GradientBoostingClassifier(n_estimators=100, max_depth=10, min_samples_split=100, min_samples_leaf=10, max_features=4)
    model.fit(X_train, y_train)
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("gbdt_res.csv",index=False,sep=' ')
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls


def train_and_test_lgb(train_df, test_df, res_df, target, features, categorical):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    
    
    
#     test['lgb_predict'] = clf.predict_proba(test[features],)[:, 1]
#     print(log_loss(test[target], test['lgb_predict']))

    
    model = lgb.LGBMClassifier(num_leaves=10, max_depth=7, n_estimators=80, n_jobs=4)
    model.fit(X_train, y_train, feature_name=features,categorical_feature=categorical)
    
    ft_w = pd.DataFrame(model.feature_importances_, columns=["weights"], index = list(X_train.columns))
    print ft_w
#     fig, ax = plt.subplots(figsize=(12,18))
#     xgboost.plot_importance(model,ax=ax)
#     plt.show()
    
    train_lls = log_loss(y_train,model.predict_proba(X_train))
    test_lls = log_loss(y_test,model.predict_proba(X_test))
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("lgb_res.csv",index=False,sep=' ')
        
    del train_df
    del test_df
    
    del X_train
    del y_train
    del X_test
    del y_test
    
    return train_lls, test_lls


def train_and_test_FFM(train_df, test_df, res_df):
    global cols_tool
    
    model = FFM(target=cols_tool.target, categorical=cols_tool.categorical, numerical=cols_tool.numerical, listype=cols_tool.listype,
                                        reg_param = 0.0004,
                                        k = 4,
                                        iter_max = 500,
                                        learing_rate = 0.1,
                                        threads = 7,
                                        auto_stop = False,
                                        quiet = False,
                                        no_norm = False)


    
    model.fit(train_df, test_df)
    
    train_lls = log_loss(train_df["is_trade"],model.predict_proba(train_df))
    test_lls = log_loss(test_df["is_trade"],model.predict_proba(test_df))
    
    
    if type(res_df) != type(None):
        del res_df["is_trade"]
        res_df = res_df.reset_index(drop=True)
        instance_id_list = res_df[["instance_id"]]
        del res_df["instance_id"]
        predicted_score = pd.DataFrame(model.predict_proba(res_df))
        
        instance_id_list["predicted_score"] = predicted_score[1]
        instance_id_list.to_csv("ffm.csv",index=False,sep=' ')
        
    
    return train_lls, test_lls


def select_best_features(df, precent=0.8):
    from sklearn.feature_selection import SelectKBest,chi2 
    df = df.copy().replace(-1, 0).fillna(0)
    
    for u in df.columns:
        df[df[u] == -1] = 0

    X_train = df.copy()
    del X_train['is_trade']
    y_train = df['is_trade']
    
    c = int(len(X_train.columns) * precent)
    skb = SelectKBest(chi2,k=c)
    skb.fit_transform(X_train,y_train)
    
    old_fea = list(X_train.columns)
    new_fea = []
    for idx, b in enumerate(list(skb.get_support())):
        if b:
            new_fea.append(old_fea[idx])
        else:
            print("Filter feature:%s" % old_fea[idx])
    return new_fea

# 开搞！

In [574]:

def dim_reduce_topn(df, dim, n, v=1):
    df2 = pd.DataFrame(df[dim].value_counts()/len(df))
    m = dict(df2.nlargest(n, dim)[dim])
    df[dim] = df[dim].map(lambda x: x if x in m else v)
    return df

def dim_reduce_rate(df, dim, rate, v=1):
    df2 = pd.DataFrame(df[dim].value_counts()/len(df))
    m = dict(df2[df2[dim] > rate][dim])
    df[dim] = df[dim].map(lambda x: x if x in m else v)
    return df

def dim_reduce_cnt(df, dim, cnt, v=1):
    df2 = pd.DataFrame(df[dim].value_counts())
    m = dict(df2[df2[dim] > cnt][dim])
    df[dim] = df[dim].map(lambda x: x if x in m else v)
    return df

def bucketizer_linear(df, key, bins, cover=False):
    nkey="*BUK*_"+key
    df[nkey] = np.nan
    idxer = df[key].values < bins[0]
    print idxer
    b = 1
    df.loc[idxer, nkey] = b
    print "x < %s : %s" % (bins[0], b)
    b += 1
    start = bins[0]
    for end in bins[1:]:
        idxer = np.logical_and(start <= df[key].values, df[key].values < end)
        df.loc[idxer, nkey] = b
        
        print "%s <= x < %s : %s" % (start, end, b)
        b+=1
        
        start = end
    
    print "%s <= x : %s" % (bins[-1], b)
    idxer = bins[-1] <= df[key].values
    df.loc[idxer, nkey] = b
    if cover:
        df[key] = df[nkey]
        df.drop(nkey)
    return df



    
# tmp_train_df["prop_len"] = tmp_train_df["item_property_list"].map(lambda x: len(x))
# tmp_train_df[["prop_len"]].describe()

In [569]:
tmp_df = train_df.copy()
# tmp_df = dim_reduce_topn(tmp_df, "city_review_cnt", 8, 666)
bucketizer_linear(tmp_df, "city_review_cnt", [1,2,3,4,5,6,7,8,9,12], True)
for i in xrange(60):
    print i, len(tmp_df[tmp_df["city_review_cnt"] == i])

print 666, len(tmp_df[tmp_df["city_review_cnt"] == 666])

/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less


[ True  True  True ...,  True  True  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x < 7 : 7
7 <= x < 8 : 8
8 <= x < 9 : 9
9 <= x < 12 : 10
12 <= x : 11


/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in greater_equal
/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in less
/Users/yuhua/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in greater_equal


0 0
1 350994
2 82722
3 30298
4 13823
5 7076
6 4011
7 2383
8 1477
9 933
10 1412
11 1097
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
666 0


In [575]:
#这块一般不怎么需要改了，拿最后一天做验证
#注意下面验证版和测试版，测试版是我们自己玩的，验证版是扔线上的，通过注释不同的跑就行了


# %run BaseFrame.py
% run ../util/time_series_split.py

tmp_train_df = train_df.copy().fillna(-1)
tmp_train_df["item_category_1"] = tmp_train_df["item_category_1"].astype('int')
tmp_train_df["item_category_2"] = tmp_train_df["item_category_2"].astype('int')
# tmp_train_df["context_page_id"] = tmp_train_df["context_page_id"]-4000


#reduce
print len(dim_reduce_rate(tmp_train_df, "shop_id", 0.01)["shop_id"].unique())
print len(dim_reduce_rate(tmp_train_df, "item_city_id", 0.01)["item_city_id"].unique())
print len(dim_reduce_rate(tmp_train_df, "item_brand_id", 0.01)["item_brand_id"].unique())
print len(dim_reduce_topn(tmp_train_df, "item_id", 30)["item_id"].unique())

dim_reduce_topn(tmp_train_df, "item_review_cnt", 3, 666)
dim_reduce_topn(tmp_train_df, "cate_review_cnt", 8, 666)
dim_reduce_topn(tmp_train_df, "brand_review_cnt", 3, 666)


bucketizer_linear(tmp_train_df, "price_review_cnt", [1,2,3,4,5,6,7,8,9,10,12], True)
bucketizer_linear(tmp_train_df, "sales_review_cnt", [1,2,3,4,5,6], True)
dim_reduce_topn(tmp_train_df, "pv_review_cnt", 6, 666)
dim_reduce_topn(tmp_train_df, "collected_review_cnt", 6, 666)
bucketizer_linear(tmp_train_df, "city_review_cnt", [1,2,3,4,5,6,7,8,9,12], True)

# print len(dim_reduce_rate(tmp_train_df, 'user_gender_id', 0.1)["user_gender_id"].unique())
# print len(dim_reduce_topn(tmp_train_df, 'context_page_id', 5)["context_page_id"].unique())

# #bucketizer_linear
# for c in [  
#                 ('item_price_level', [4, 6, 8]),
#                 ('item_sales_level', [3, 6, 9, 12, 15]),
#                 ('item_collected_level', [3, 6, 9, 12, 15]),
#                 ('item_pv_level', [4, 8, 12, 16, 20]),
#                 ('user_star_level', [3003, 3005, 3007, 3009]),
#                 ('shop_star_level', [5003, 5007, 5011, 5015, 5019]),
#                 ('shop_review_num_level', [4, 8, 12, 16, 20]),
#                 ("hour", [2, 7, 12, 17, 19])
#             ]:
    
#     bk = c[1]
#     c = c[0]
    
#     tmp_train_df = bucketizer_linear(tmp_train_df, c, bk, True)
#     print "-"* 20

# tmp_train_df["user_gender_id*shop_review_positive_rate"] = tmp_train_df["user_gender_id"] * (1+tmp_train_df["shop_review_positive_rate"])

# tmp_train_df["user_gender_id*shop_star_level"] = tmp_train_df["user_gender_id"] * (1+tmp_train_df["shop_star_level"])


cols_tool.fit(train_df)

categorical_one_hot_cols = cols_tool.get_onehoted_cols("categorical")
listype_one_hot_cols = cols_tool.get_onehoted_cols("listype")

index_col = "instance_id"


categorical_cols = cols_tool.get_raw_categorical_cols()
listype_cols = cols_tool.get_raw_listype_cols()
numerical_cols = cols_tool.get_raw_numerical_cols()
target_col = cols_tool.get_raw_target_col()


print("org dataset = %s" % len(tmp_train_df))
print(tmp_train_df["day"].value_counts())


# tmp_train_df = featProc.balance_pos_neg_sample(tmp_train_df, (1,10))
# print "sampling dataset =", len(tmp_train_df)
# print tmp_train_df["day"].value_counts()

# 抽样
# tmp_train_df = tmp_train_df.sample(n=10000, random_state=666)
# print "sampling dataset =", len(tmp_train_df)

print(tmp_train_df["day"].value_counts())

#测试版
valid_df = tmp_train_df.loc[tmp_train_df["day"]==6]
tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<6]
# tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]>0]
res_df=valid_df.copy()

# #线上验证版
# valid_df = tmp_train_df.loc[tmp_train_df["day"]==6]
# res_df = tmp_train_df.loc[tmp_train_df["day"]==7]
# tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<=6]

print("res dataset = %s" % len(res_df))
print("valid dateset = %s" % len(valid_df))
print("train dateset = %s" % len(tmp_train_df))




use_fea1=[  
                'item_price_level',
                'item_sales_level',
                'item_collected_level',
                'item_pv_level',
                'user_gender_id',
                'user_age_level',
                'user_occupation_id',
                'user_star_level',
                'context_page_id',
                'shop_review_num_level',
                'shop_star_level',
                "weekday",
                "hour",
    
                'item_category_1',
                'item_category_2',
                
                "item_brand_id",
                "item_city_id",
                "item_id",
                "shop_id",
    
                "item_review_cnt",
                "cate_review_cnt",
    
                'category_sim',
                'property_sim',
                'hit_cate_cnt',
                'hit_cate_sim',
                'item_property_richness',
                'predict_richness',
                
                'shop_review_positive_rate',
                'shop_score_service',
                'shop_score_delivery',
                'shop_score_description',
            ]

use_fea2=[  
                'item_price_level',
                'item_sales_level',
                'item_collected_level',
                'item_pv_level',
                'user_gender_id',
                'user_age_level',
                'user_occupation_id',
                'user_star_level',
                'context_page_id',
                'shop_review_num_level',
                'shop_star_level',
                "weekday",
                "hour",
    
                'item_category_1',
                'item_category_2',
                
                "item_brand_id",
                "item_city_id",
                "item_id",
                "shop_id",
    
                "item_review_cnt",
                "cate_review_cnt",
    
                'category_sim',
                'property_sim',
                'hit_cate_cnt',
                'hit_cate_sim',
                'item_property_richness',
                'predict_richness',
                
                'shop_review_positive_rate',
                'shop_score_service',
                'shop_score_delivery',
                'shop_score_description'
            ]




# tmp_train_df = tmp_train_df[on_cols]
# valid_df = valid_df[on_cols]


# on_cols = [target_col]+numerical_cols+categorical_one_hot_cols+listype_one_hot_cols
# # on_cols = [target_col]+numerical_cols
# # +categorical_one_hot_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_lr(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("LR %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))



# on_cols = [target_col]+categorical_cols+numerical_cols
# # on_cols = [target_col]+select_best_features(tmp_train_df[on_cols], 0.8)
# train_lls, test_lls = train_and_test_randomforest(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("XGBoost %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))



# on_cols = [target_col]+categorical_cols+numerical_cols
# on_cols = [target_col]+select_best_features(tmp_train_df[on_cols], 0.95)
# train_lls, test_lls = train_and_test_randomforest(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("XGBoost %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))




# +listype_one_hot_cols
# on_cols = ffea(on_cols)



# use_fea = use_fea1
# train_lls, test_lls = train_and_test_gbdt(tmp_train_df[use_fea+[target]], valid_df[use_fea+[target]], None)
# print("GBDT %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[use_fea].columns), train_lls, test_lls))

# use_fea = use_fea2
# train_lls, test_lls = train_and_test_gbdt(tmp_train_df[use_fea+[target]], valid_df[use_fea+[target]], None)
# print("GBDT %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[use_fea].columns), train_lls, test_lls))





# use_fea = use_fea1
# train_lls, test_lls, test_a_lls, test_b_lls = train_and_test_randomforest(tmp_train_df[use_fea+[target]], valid_df[use_fea+[target]], None)
# print("RandomForest %s dimension TrainLogLoss = %s | TestLogLoss = %s ; a = %s ; b = %s " % (len(tmp_train_df[use_fea].columns), train_lls, test_lls, test_a_lls, test_b_lls))

use_fea = use_fea2

train_lls, test_lls, test_a_lls, test_b_lls = train_and_test_randomforest(tmp_train_df[use_fea+[target]], valid_df[use_fea+[target]], None)
print("RandomForest %s dimension TrainLogLoss = %s | TestLogLoss = %s ; a = %s ; b = %s " % (len(tmp_train_df[use_fea].columns), train_lls, test_lls, test_a_lls, test_b_lls))

    
for i in [
                "brand_review_cnt",
                'price_review_cnt',
                'sales_review_cnt',
                'pv_review_cnt',
                'collected_review_cnt',
                'city_review_cnt']:
    use_fea.append(i)
    train_lls, test_lls, test_a_lls, test_b_lls = train_and_test_randomforest(tmp_train_df[use_fea+[target]], valid_df[use_fea+[target]], None)
    print("RandomForest %s dimension TrainLogLoss = %s | TestLogLoss = %s ; a = %s ; b = %s " % (len(tmp_train_df[use_fea].columns), train_lls, test_lls, test_a_lls, test_b_lls))



# on_cols = [target_col]+categorical_cols+numerical_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_gbdt(tmp_train_df[on_cols], valid_df[on_cols], res_df[on_cols+[index_col]])
# print("GBDT %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))


# train_lls, test_lls = train_and_test_combine(tmp_train_df[on_cols], valid_df[on_cols])
# print "Combine TrainLogLoss = %s | TestLogLoss = %s" % (train_lls, test_lls)

# on_cols = [target_col]+numerical_cols+categorical_one_hot_cols+listype_one_hot_cols
# train_lls, test_lls = train_and_test_FFM(tmp_train_df, valid_df)
# print "FFM TrainLogLoss = %s | TestLogLoss = %s" % (train_lls, test_lls)

# train_lls, test_lls = train_and_test_FFM(tmp_train_df, valid_df, res_df)
# print("LR %s dimension TrainLogLoss = %s | TestLogLoss = %s" % (len(tmp_train_df[on_cols].columns), train_lls, test_lls))



5
12
7
31
[ True  True  True ...,  True  True  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x < 7 : 7
7 <= x < 8 : 8
8 <= x < 9 : 9
9 <= x < 10 : 10
10 <= x < 12 : 11
12 <= x : 12
[ True  True  True ...,  True  True  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x : 7
[ True  True  True ...,  True  True  True]
x < 1 : 1
1 <= x < 2 : 2
2 <= x < 3 : 3
3 <= x < 4 : 4
4 <= x < 5 : 5
5 <= x < 6 : 6
6 <= x < 7 : 7
7 <= x < 8 : 8
8 <= x < 9 : 9
9 <= x < 12 : 10
12 <= x : 11
org dataset = 496509
0    78268
3    71199
1    70931
2    68387
4    68318
5    63614
6    57421
7    18371
Name: day, dtype: int64
0    78268
3    71199
1    70931
2    68387
4    68318
5    63614
6    57421
7    18371
Name: day, dtype: int64
res dataset = 57421
valid dateset = 57421
train dateset = 420717


[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:   31.4s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:  1.3min finished


                            weights
item_price_level           0.138907
item_review_cnt            0.090428
shop_score_delivery        0.089820
shop_score_service         0.070373
item_sales_level           0.065542
shop_score_description     0.054386
hour                       0.050593
item_category_1            0.048341
item_property_richness     0.042400
shop_review_positive_rate  0.033267
item_city_id               0.033106
user_age_level             0.028316
user_star_level            0.028187
property_sim               0.026896
predict_richness           0.021989
shop_review_num_level      0.019324
item_collected_level       0.018758
shop_star_level            0.018544
cate_review_cnt            0.015940
category_sim               0.015836
context_page_id            0.015807
hit_cate_sim               0.014098
item_pv_level              0.012271
weekday                    0.010041
item_id                    0.008351
user_occupation_id         0.007447
item_category_2            0

[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    1.7s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    3.5s finished
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.2s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.3s finished


RandomForest 31 dimension TrainLogLoss = 0.0850756163041 | TestLogLoss = 0.0819560555141 ; a = 0.08100225248 ; b = 0.0823648066602 


...

RandomForest 36 dimension TrainLogLoss = 0.0851209531427 | TestLogLoss = 0.0820872183838 ; a = 0.0811241567189 ; b = 0.08249992024 


[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:   31.4s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:  1.1min finished


                            weights
item_price_level           0.136589
shop_score_delivery        0.076487
shop_score_service         0.073842
item_review_cnt            0.070161
item_sales_level           0.064616
shop_score_description     0.053579
item_category_1            0.050530
hour                       0.045307
item_property_richness     0.042475
item_city_id               0.032069
shop_review_positive_rate  0.030251
user_star_level            0.026535
user_age_level             0.025777
property_sim               0.024977
predict_richness           0.021422
shop_review_num_level      0.019687
item_collected_level       0.018331
brand_review_cnt           0.018287
shop_star_level            0.018170
context_page_id            0.015740
category_sim               0.014522
item_pv_level              0.013621
hit_cate_sim               0.013491
cate_review_cnt            0.011888
weekday                    0.009817
price_review_cnt           0.008833
item_id                    0

[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    1.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.3s finished


RandomForest 37 dimension TrainLogLoss = 0.0851848928408 | TestLogLoss = 0.0820266980494 ; a = 0.080969080752 ; b = 0.0824799751201 


In [ ]:
tmp_df = train_df.copy()
dim_reduce_topn(tmp_df, "item_review_cnt", 3, 666)
dim_reduce_topn(tmp_df, "cate_review_cnt", 8, 666)


for i in xrange(10):
    print i, len(tmp_df[tmp_df["item_review_cnt"] == i])

print 666, len(tmp_df[tmp_df["item_review_cnt"] == 666])

print " "
for i in xrange(10):
    print i, len(tmp_df[tmp_df["cate_review_cnt"] == i])
print 666, len(tmp_df[tmp_df["cate_review_cnt"] == 666])

In [ ]:
tmp_train_df = train_df.copy()
valid_df = gen_valid_set(tmp_train_df)
# valid_df = gen_valid_set_FULL(tmp_train_df)

tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<6]
res_df=valid_df.copy()


print len(tmp_train_df),  len(valid_df)


categorical_one_hot_cols = cols_tool.get_onehoted_cols("categorical", tmp_train_df)
listype_one_hot_cols = cols_tool.get_onehoted_cols("listype", tmp_train_df)

index_col = "instance_id"

categorical_cols = cols_tool.get_raw_categorical_cols()
listype_cols = cols_tool.get_raw_listype_cols()
numerical_cols = cols_tool.get_raw_numerical_cols()
target_col = cols_tool.get_raw_target_col()

In [ ]:
print len(valid_df)
print valid_df["is_trade"].value_counts()

In [ ]:
on_cols = [target_col]+numerical_cols+categorical_one_hot_cols#+listype_one_hot_cols
# print on_cols
train_lls, test_lls = train_and_test_lr(tmp_train_df[on_cols], valid_df[on_cols], None)
print("LR TrainLogLoss = %s | TestLogLoss = %s" % (train_lls, test_lls))


In [ ]:
X_test = valid_df.copy()
del X_test['is_trade']
y_test = valid_df.copy()['is_trade']

import random
from sklearn.model_selection import train_test_split

for i in xrange(1000000):
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=i)
    zero = []
    for j in xrange(len(y_val_a)):
        f = float('0.0000000000000%s' % random.randint(1,99))
        zero.append(f)
    lls = log_loss(y_val_a, zero)
    llsdiff = abs(0.31284 - lls)
    y_val_a = pd.DataFrame(y_val_a)
    l = float(len(y_val_a))
    
    if llsdiff < 0.0001:
        print i, "[HIT]", llsdiff, lls, len(y_val_a[y_val_a["is_trade"] == 0])/l, len(y_val_a[y_val_a["is_trade"] == 1])/l
    else:
        print i, llsdiff, lls, len(y_val_a[y_val_a["is_trade"] == 0])/l, len(y_val_a[y_val_a["is_trade"] == 1])/l
        


In [ ]:

t = pd.read_table('/Users/yuhua/res1.csv',sep=' ')
log_loss(a, t["predicted_score"])


In [ ]:

tmp_train_df = train_df.copy().replace(-1, np.nan)

#one_hot全集 0.08450
#one_hot-noid，无id类特征 0.08330
#one_hot-city，仅仅有city id 
#one_hot-noid_exp，没有id和exp类特征 0.08368
#one_hot-valid-noid_prop，没有id，prop类特征 
#one_hot-hash 使用hash降维的id
#one_hot-reduce使用reduce降维

%run ffmyh.py
%run FeatureProcess.py




# #hash
# tmp_train_df["item_brand_id"] = tmp_train_df["item_brand_id"] % 15
# tmp_train_df["item_city_id"] = tmp_train_df["item_city_id"] % 15
# tmp_train_df["item_id"] = tmp_train_df["item_id"] % 15
# tmp_train_df["shop_id"] = tmp_train_df["shop_id"] % 15


#reduce
print len(dim_reduce_rate(tmp_train_df, "shop_id", 0.01)["shop_id"].unique())
print len(dim_reduce_rate(tmp_train_df, "item_city_id", 0.01)["item_city_id"].unique())
print len(dim_reduce_rate(tmp_train_df, "item_brand_id", 0.01)["item_brand_id"].unique())
print len(dim_reduce_topn(tmp_train_df, "item_id", 30)["item_id"].unique())

print len(dim_reduce_rate(tmp_train_df, 'user_gender_id', 0.1)["user_gender_id"].unique())
print len(dim_reduce_rate(tmp_train_df, 'context_page_id', 0.05)["context_page_id"].unique())



dim_reduce_topn(tmp_train_df, "item_review_cnt", 3, 666)
dim_reduce_topn(tmp_train_df, "cate_review_cnt", 8, 666)
dim_reduce_topn(tmp_train_df, "brand_review_cnt", 3, 666)
bucketizer_linear(tmp_train_df, "price_review_cnt", [1,2,3,4,5,6,7,8,9,10,12], True)
bucketizer_linear(tmp_train_df, "sales_review_cnt", [1,2,3,4,5,6], True)
dim_reduce_topn(tmp_train_df, "pv_review_cnt", 6, 666)
dim_reduce_topn(tmp_train_df, "collected_review_cnt", 6, 666)
bucketizer_linear(tmp_train_df, "city_review_cnt", [1,2,3,4,5,6,7,8,9,12], True)


# #bucketizer_linear
# for c in [  
#                 ('item_price_level', [4, 6, 8]),
#                 ('item_sales_level', [3, 6, 9, 12, 15]),
#                 ('item_collected_level', [3, 6, 9, 12, 15]),
#                 ('item_pv_level', [4, 8, 12, 16, 20]),
#                 ('user_star_level', [3003, 3005, 3007, 3009]),
#                 ('shop_star_level', [5003, 5007, 5011, 5015, 5019]),
#                 ('shop_review_num_level', [4, 8, 12, 16, 20]),
#                 ("hour", [2, 7, 12, 17, 19])
#             ]:
    
#     bk = c[1]
#     c = c[0]
    
#     tmp_train_df = bucketizer_linear(tmp_train_df, c, bk, True)
#     print "-"* 20


# cols_tool = filter_on_cols(target, categorical, numerical, listype)
# cols_tool.fit(tmp_train_df)

# on_cols = [target]+cols_tool["categorical"]+numerical
# tmp_train_df = select_best_features(tmp_train_df[on_cols], 0.9)
    
fpfp = FeatureProcess(target=target, categorical=categorical, numerical=numerical, listype = listype)
# for n in numerical:
#     fpfp.addLayeringOrgKeys(n)
#一定要fit一个全集
fpfp.fit(tmp_train_df)


#观察FFM权重
model = FFM(fpfp)
    
model.fit_schema()
print "\nPARSE_RES\n"
model.parse_model("./tmp.txt", "./ffm/reduce-k.model")


# print("org dataset = %s" % len(tmp_train_df))
# print(tmp_train_df["day"].value_counts())


# print(tmp_train_df["day"].value_counts())

# #测试版
# valid_df = tmp_train_df.loc[tmp_train_df["day"]==6]
# tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<6]
# res_df=valid_df.copy()


# #线上验证版
# valid_df = tmp_train_df.loc[tmp_train_df["day"]==6]
# res_df = tmp_train_df.loc[tmp_train_df["day"]==7]
# tmp_train_df = tmp_train_df.loc[tmp_train_df["day"]<=6]

# model = FFM(fpfp)
# model.set_exists_model("reduce-k.model")
# model.predict(valid_df)

# fpfp.toFFMData(tmp_train_df, "one_hot-train-reduce-k.txt")
# fpfp.toFFMData(valid_df, "one_hot-valid-reduce-k.txt")
# fpfp.toFFMData(res_df, "one_hot-res-reduce-k.txt")

# model = FFM(fpfp,
#             reg_param = 0.00001,
#             k = 4,
#             iter_max = 500,
#             learing_rate = 0.01,
#             threads = 7,
#             auto_stop = False,
#             quiet = False)


    
# # model.fit(tmp_train_df, valid_df)

# # train_lls = log_loss(train_df["is_trade"],model.predict_proba(train_df))
# # test_lls = log_loss(test_df["is_trade"],model.predict_proba(test_df))

# # print train_lls, test_lls


In [572]:
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble.forest:

class RandomForestClassifier(ForestClassifier)
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and use averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |      Note: this parameter is tree-specific.
 |  
 |  max_features : int, fl

- | XGboost | 增减 | LR |增减
- | :-: | :-: | :-: | :-:
原始特征 | 0.0829350458274 | / | 0.0829428179401 | /
原始特征+时间 | 0.0828004994761 | +++ | 0.082869098617 | +
同上+按天计算交易率 | 0.0828218434014 | - | 0.0827914564667 | ++
同上+predict_prop_cate相似度 | 0.082778758021 | ++ | 0.082798919764 | --
同上+cate prop丰富度 | 0.0828356414143 | -- | 0.0828009736655 | -
同上+cate/prop list OneHot | 0.0828356414143 | / | 0.0827979171146 | +